In [ ]:
import copy
import sys
import matplotlib.pyplot as plt

import sympy as sp
import numpy as np
from IPython.display import display, Markdown

## Symbolic Calculations

In [ ]:
x, y, theta = sp.symbols(r'x, y, \theta')
xd, yd, thetad = sp.symbols(r'x_d, y_d, \theta_d')
u1, u2 = sp.symbols(r'u_1, u_2')
Q1, Q2, Q3 = sp.symbols(r'Q_1, Q_2, Q_3')
R1, R2 = sp.symbols(r'R_1, R_2')

In [ ]:
xdot = sp.Matrix([sp.cos(theta)*u1, sp.sin(theta)*u1, u2])
left = r'\dot{\vec{x}}='
display(Markdown(f'${left}{sp.latex(xdot)}$'))
# print(sp.latex(xdot))

$\dot{\vec{x}}=\left[\begin{matrix}u_{1} \cos{\left(\theta \right)}\\u_{1} \sin{\left(\theta \right)}\\u_{2}\end{matrix}\right]$

In [ ]:
x_vec = sp.Matrix([x, y, theta])
xd_vec = sp.Matrix([xd, yd, thetad])
u_vec = sp.Matrix([u1, u2])

x_left = r'\vec{x}='
u_left = r'\vec{u}='
display(Markdown(f'${x_left}{sp.latex(x_vec)}$'))
display(Markdown(f'${u_left}{sp.latex(u_vec)}$'))

$\vec{x}=\left[\begin{matrix}x\\y\\\theta\end{matrix}\right]$

$\vec{u}=\left[\begin{matrix}u_{1}\\u_{2}\end{matrix}\right]$

In [ ]:
A_mat = xdot.jacobian(x_vec)
B_mat = xdot.jacobian(u_vec)

A_left = r'A='
B_left = r'B='

display(Markdown(f'${A_left}{sp.latex(A_mat)}$'))
display(Markdown(f'${B_left}{sp.latex(B_mat)}$'))

$A=\left[\begin{matrix}0 & 0 & - u_{1} \sin{\left(\theta \right)}\\0 & 0 & u_{1} \cos{\left(\theta \right)}\\0 & 0 & 0\end{matrix}\right]$

$B=\left[\begin{matrix}\cos{\left(\theta \right)} & 0\\\sin{\left(\theta \right)} & 0\\0 & 1\end{matrix}\right]$

In [ ]:
Qx = sp.diag(Q1, Q2, Q3)
Ru = sp.diag(R1, R2)

qx_left = 'Q_x='
ru_left = 'R_u='

display(Markdown(f'${qx_left}{sp.latex(Qx)}$'))
display(Markdown(f'${ru_left}{sp.latex(Ru)}$'))

$Q_x=\left[\begin{matrix}Q_{1} & 0 & 0\\0 & Q_{2} & 0\\0 & 0 & Q_{3}\end{matrix}\right]$

$R_u=\left[\begin{matrix}R_{1} & 0\\0 & R_{2}\end{matrix}\right]$

In [ ]:
l = (x_vec - xd_vec).T @ Qx @ (x_vec - xd_vec) + u_vec.T @ Ru @ u_vec

left_l = 'l(x, u)='
display(Markdown(f'${left_l}{sp.latex(l[0])}$'))

$l(x, u)=Q_{1} \left(x - x_{d}\right)^{2} + Q_{2} \left(y - y_{d}\right)^{2} + Q_{3} \left(\theta - \theta_{d}\right)^{2} + R_{1} u_{1}^{2} + R_{2} u_{2}^{2}$

In [ ]:
dldx = l.jacobian(x_vec).T
dldu = l.jacobian(u_vec).T

display(dldx)
display(dldu)

Matrix([
[          Q_1*(2*x - 2*x_d)],
[          Q_2*(2*y - 2*y_d)],
[Q_3*(2*\theta - 2*\theta_d)]])

Matrix([
[2*R_1*u_1],
[2*R_2*u_2]])

## Numeric Calculations

In [ ]:
# Clean environment
# sys.modules[__name__].__dict__.clear()

In [ ]:
### define parameters

dt = 0.1
x0 = np.array([0.0, 0.0, np.pi/2.0])
tsteps = 63
init_u_traj = np.tile(np.array([1.0, -0.5]), reps=(tsteps,1))

Q_x = np.diag([10.0, 10.0, 2.0])
R_u = np.diag([4.0, 2.0])
P1 = np.diag([20.0, 20.0, 5.0])

Q_z = np.diag([5.0, 5.0, 1.0])
R_v = np.diag([2.0, 1.0])

In [ ]:
def dyn(xt, ut):
    # xdot = np.zeros(3)  # replace this
    theta = xt[2]
    u1 = ut[0]
    u2 = ut[1]
    x1dot = np.cos(theta) * u1
    x2dot = np.sin(theta) * u1
    x3dot = u2

    xdot = np.array([x1dot, x2dot, x3dot])
    return xdot


def get_A(t, xt, ut):
    theta = xt[2]
    u1 = ut[1]
    A_mat = np.zeros((3, 3))  # replace this
    A_mat[0, 2] = -np.sin(theta) * u1
    A_mat[1, 2] = np.cos(theta) * u1
    return A_mat


def get_B(t, xt, ut):
    theta = xt[2]
    B_mat = np.zeros((3, 2))  # replace this
    B_mat[0, 0] = np.cos(theta)
    B_mat[1, 0] = np.sin(theta)
    B_mat[2, 1] = 1
    return B_mat


def get_xd(t):
    xd = np.array([4.0 / (2.0 * np.pi) * t, 0.0, np.pi / 2.0])
    return xd


def step(xt, ut, dt):
    # xt_new = xt + dt * dyn(xt, ut)  # recommended: replace it with RK4 integration
    k1 = dt * dyn(xt, ut)
    k2 = dt * dyn(xt + k1 / 2.0, ut)
    k3 = dt * dyn(xt + k2 / 2.0, ut)
    k4 = dt * dyn(xt + k3, ut)

    xt_new = xt + 1.0 / 6.0 * (k1 + 2.0 * k2 + 2.0 * k3 + k4)
    return xt_new


def traj_sim(x0, ulist):
    tsteps = ulist.shape[0]
    x_traj = np.zeros((tsteps, 3))
    xt = copy.deepcopy(x0)
    for t in range(tsteps):
        xt_new = step(xt, ulist[t], dt)
        x_traj[t] = copy.deepcopy(xt_new)
        xt = copy.deepcopy(xt_new)
    return x_traj


def loss(t, xt, ut):
    # xd = np.array([2.0 * t / np.pi, 0.0, np.pi / 2.0])  # desired system state at time t
    xd = get_xd(t)

    x_diff = xt - xd
    # x_loss = 0.0  # replace this
    x_loss = x_diff.T @ Q_x @ x_diff
    # u_loss = 0.0  # replace this
    u_loss = ut.T @ R_u @ ut

    return x_loss + u_loss


def dldx(t, xt, ut):
    # xd = np.array([2.0 * t / np.pi, 0.0, np.pi / 2.0])
    xd = get_xd(t)
    qlist = np.diag(Q_x)

    # dvec = np.zeros(3)  # replace this
    dx = xt - xd
    dvec = qlist * 2 * dx
    return dvec


def dldu(t, xt, ut):
    # dvec = np.zeros(2)  # replace this
    rlist = np.diag(R_u)
    dvec = 2 * rlist * ut
    return dvec

# xd_vec

In [ ]:
def ilqr_iter(x0, u_traj):
    """
    :param x0: initial state of the system
    :param u_traj: current estimation of the optimal control trajectory
    :return: the descent direction for the control
    """
    # forward simulate the state trajectory
    x_traj = traj_sim(x0, u_traj)

    # compute other variables needed for specifying the dynamics of z(t) and p(t)
    A_list = np.zeros((tsteps, 3, 3))
    B_list = np.zeros((tsteps, 3, 2))
    a_list = np.zeros((tsteps, 3))
    b_list = np.zeros((tsteps, 2))
    for t_idx in range(tsteps):
        t = t_idx * dt
        A_list[t_idx] = get_A(t, x_traj[t_idx], u_traj[t_idx])
        B_list[t_idx] = get_B(t, x_traj[t_idx], u_traj[t_idx])
        a_list[t_idx] = dldx(t, x_traj[t_idx], u_traj[t_idx])
        b_list[t_idx] = dldu(t, x_traj[t_idx], u_traj[t_idx])

    xd_T = np.array([
        2.0*(tsteps-1)*dt / np.pi, 0.0, np.pi/2.0
    ])  # desired terminal state
    p1 = np.zeros(3)  # replace it to be the terminal condition p(T)

    def zp_dyn(t, zp):
        t_idx = (t/dt).astype(int)
        At = A_list[t_idx]
        Bt = B_list[t_idx]
        at = a_list[t_idx]
        bt = b_list[t_idx]

        M_11 = np.zeros((3,3))  # replace this
        M_12 = np.zeros((3,3))  # replace this
        M_21 = np.zeros((3,3))  # replace this
        M_22 = np.zeros((3,3))  # replace this
        dyn_mat = np.block([
            [M_11, M_12],
            [M_21, M_22]
        ])

        m_1 = np.zeros(3)  # replace this
        m_2 = np.zeros(3)  # replace this
        dyn_vec = np.hstack([m_1, m_2])

        return dyn_mat @ zp + dyn_vec

    # this will be the actual dynamics function you provide to solve_bvp,
    # it takes in a list of time steps and corresponding [z(t), p(t)]
    # and returns a list of [zdot(t), pdot(t)]
    def zp_dyn_list(t_list, zp_list):
        list_len = len(t_list)
        zp_dot_list = np.zeros((6, list_len))
        for _i in range(list_len):
            zp_dot_list[:,_i] = zp_dyn(t_list[_i], zp_list[:,_i])
        return zp_dot_list

    # boundary condition (inputs are [z(0),p(0)] and [z(T),p(T)])
    def zp_bc(zp_0, zp_T):
        return np.zeros(6)  # replace this

    ### The solver will say it does not converge, but the returned result
    ### is numerically accurate enough for our use
    zp_traj = np.zeros((tsteps,6))  # replace this by using solve_bvp

    z_traj = zp_traj[:,:3]
    p_traj = zp_traj[:,3:]

    v_traj = np.zeros((tsteps, 2))
    for _i in range(tsteps):
        At = A_list[_i]
        Bt = B_list[_i]
        at = a_list[_i]
        bt = b_list[_i]

        zt = z_traj[_i]
        pt = p_traj[_i]

        vt = np.zeros(2)  # replace this
        v_traj[_i] = vt

    return v_traj

In [ ]:
# Start iLQR iterations here

u_traj = init_u_traj.copy()
for iter in range(10):
    # forward simulate the current trajectory
    x_traj = traj_sim(x0, u_traj)

    # visualize the current trajectory
    fig, ax = plt.subplots(1, 1)
    ax.set_title('Iter: {:d}'.format(iter))
    ax.set_aspect('equal')
    ax.set_xlim(-0.2, 4.2)
    ax.set_ylim(-0.2, 2.2)
    ax.plot(x_traj[:,0], x_traj[:,1], linestyle='-', color='C0')
    plt.show()
    plt.close()

    # get descent direction
    v_traj = ilqr_iter(x0, u_traj)

    # Armijo line search parameters
    gamma = 1.0  # initial step size
    alpha = 1e-04
    beta = 0.5

    ### Implement Armijo line search here to update step size gamma

    # update control for the next iteration
    u_traj += gamma * v_traj